In [ ]:
!pip install supervision

In [ ]:
!pip install ultralytics

In [ ]:
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1M3UuH3QNDWGiH0NmGgHtIgXXGDo_nigm' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1M3UuH3QNDWGiH0NmGgHtIgXXGDo_nigm" -O mall.mp4 && rm -rf /tmp/cookies.txt

In [ ]:
import supervision as sv
import cv2

generator = sv.get_video_frames_generator("./test.webm")
iterator = iter(generator)
frame = next(iterator)

cv2.imwrite("test.png", frame)

In [4]:
import numpy as np
import supervision as sv

MALL_VIDEO_PATH = "./test.webm"

# initialize polygon zone
polygon = np.array([
    [16, 42],[16, 998],[1652, 994],[1660, 42],[8, 34]
])

In [ ]:
video_info = sv.VideoInfo.from_video_path("./test.webm")
zone = sv.PolygonZone(polygon=polygon, frame_resolution_wh=video_info.resolution_wh)

box_annotator = sv.BoxAnnotator(thickness=4, text_thickness=4, text_scale=2)
zone_annotator = sv.PolygonZoneAnnotator(zone=zone, color=sv.Color.white(), thickness=6, text_thickness=6, text_scale=4)

In [ ]:
from ultralytics import YOLO

model = YOLO("yolov8s.pt")

In [7]:
def process_frame(frame: np.ndarray, _) -> np.ndarray:
   results = model(frame, imgsz=1280)[0]
   detections = sv.Detections.from_yolov8(results)
   detections = detections[detections.class_id == 0]
   zone.trigger(detections=detections)

   box_annotator = sv.BoxAnnotator(thickness=1, text_thickness=1, text_scale=0.10)

   labels = [f"{model.names[class_id]} {confidence:0.2f}" for _, _, confidence, class_id, _ in detections]
   frame = box_annotator.annotate(scene=frame, detections=detections, labels=labels)
   frame = zone_annotator.annotate(scene=frame)


   return frame

In [ ]:
sv.process_video(source_path=MALL_VIDEO_PATH, target_path=f"test-121-result.mp4", callback=process_frame)